In [5]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import lightgbm as lgb
import pandas as pd



import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score





In [23]:
def make_class(vv):
    if vv < 0.1:
        return 0
    elif vv < 0.2:
        return 1
    elif vv < 0.5:
        return 2
    elif vv < 1.0:
        return 3
    elif vv < 2.0:
        return 4
    elif vv < 5.0:
        return 5
    elif vv < 10.0:
        return 6
    elif vv < 20.0:
        return 7
    elif vv < 30.0:
        return 8
    else:
        return 9

In [15]:
train = pd.read_csv("train2.csv")
test = pd.read_csv("test2.csv")

In [16]:
# 장마 변수 추가

train['season'] = ((train['ef_month'] == 6) & (train['ef_day'] >= 23)) | (train['ef_month'] == 7 & (train['ef_day'] <= 26))
train['season'] = train['season'].astype(int)
test['season'] = ((test['ef_month'] == 6) & (test['ef_day'] >= 23)) | (test['ef_month'] == 7 & (test['ef_day'] <= 26))
test['season'] = test['season'].astype(int)

# https://data.kma.go.kr/climate/rainySeason/selectRainySeasonList.do


In [17]:
train

,stn4contest,dh,ef_year,ef_month,ef_day,ef_hour,v01,v02,v03,v04,v05,v06,v07,v08,v09,vv,fc_datetime,ef_datetime,season
0,1,3,2001,5,1,12,0,0,0,0,0,0,0,0,0,0.0,2001-05-01 09:00:00,2001-05-01 12:00:00,0
1,1,6,2001,5,1,15,0,0,0,0,0,0,0,0,0,0.0,2001-05-01 09:00:00,2001-05-01 15:00:00,0
2,1,9,2001,5,1,18,0,0,0,0,0,0,0,0,0,0.0,2001-05-01 09:00:00,2001-05-01 18:00:00,0
3,1,12,2001,5,1,21,0,0,0,0,0,0,0,0,0,0.0,2001-05-01 09:00:00,2001-05-01 21:00:00,0
4,1,15,2001,5,2,0,0,0,0,0,0,0,0,0,0,0.0,2001-05-01 09:00:00,2001-05-02 00:00:00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1448757,20,228,2003,10,10,9,9,7,2,2,0,0,0,0,0,0.5,2003-09-30 21:00:00,2003-10-10 09:00:00,0
1448758,20,231,2003,10,10,12,13,9,5,3,1,0,0,0,0,0.0,2003-09-30 21:00:00,2003-10-10 12:00:00,0
1448759,20,234,2003,10,10,15,13,9,5,3,1,0,0,0,0,1.9,2003-09-30 21:00:00,2003-10-10 15:00:00,0
1448760,20,237,2003,10,10,18,8,6,4,2,2,0,0,0,0,1.9,2003-09-30 21:00:00,2003-10-10 18:00:00,0


In [18]:
# fc_time 데이터 통합
def aggregate_fc_times(df):
    agg_funcs = {
        'v01': ['mean', 'max', 'min'],
        'v02': ['mean', 'max', 'min'],
        'v03': ['mean', 'max', 'min'],
        'v04': ['mean', 'max', 'min'],
        'v05': ['mean', 'max', 'min'],
        'v06': ['mean', 'max', 'min'],
        'v07': ['mean', 'max', 'min'],
        'v08': ['mean', 'max', 'min'],
        'v09': ['mean', 'max', 'min']
    }
    df_agg = df.groupby(['ef_datetime', 'stn4contest']).agg(agg_funcs)
    df_agg.columns = ['_'.join(col).strip() for col in df_agg.columns.values]

    # dh가 가장 작은 값 선택
    idx_min_dh = df.groupby(['ef_datetime', 'stn4contest'])['dh'].idxmin()
    recent_values = df.loc[idx_min_dh, ['ef_datetime', 'stn4contest', 
                                        'v01', 'v02', 'v03', 'v04', 'v05', 
                                        'v06', 'v07', 'v08', 'v09', 'dh']]
    
    # 컬럼 이름 변경 (최근 값을 나타내기 위해)
    recent_values.columns = ['ef_datetime', 'stn4contest',
                             'recent_v01', 'recent_v02', 'recent_v03', 'recent_v04', 'recent_v05', 
                             'recent_v06', 'recent_v07', 'recent_v08', 'recent_v09', 'recent_dh']
    
    
    # Aggregated Data와 Recent Values 병합
    df_merged = pd.merge(df_agg, recent_values, on=['ef_datetime', 'stn4contest'])

    
    return df_merged

train_agg = aggregate_fc_times(train)
test_agg = aggregate_fc_times(test)


In [19]:
# 시간 관련 피처 생성 (Sine/Cosine 변환)
def add_sin_cos_features(df, col):
    df[col + '_sin'] = np.sin(2 * np.pi * df[col] / df[col].max())
    df[col + '_cos'] = np.cos(2 * np.pi * df[col] / df[col].max())
    return df

# 타겟 변수 추가
def add_target_variable(df_agg, original_df):
    target = original_df.groupby(['ef_datetime', 'stn4contest'])['vv'].first().reset_index()
    df_agg = pd.merge(df_agg, target, on=['ef_datetime', 'stn4contest'], how='left')
    return df_agg

# 장마 변수 추가
def add_season(df_agg, original_df):
    target = original_df.groupby(['ef_datetime', 'stn4contest'])['season'].first().reset_index()
    df_agg = pd.merge(df_agg, target, on=['ef_datetime', 'stn4contest'], how='left')
    return df_agg


# 시간 변수 추가
def add_time_hour(df_agg, original_df):
    target = original_df.groupby(['ef_datetime', 'stn4contest'])['ef_hour'].first().reset_index()
    df_agg = pd.merge(df_agg, target, on=['ef_datetime', 'stn4contest'], how='left')
    return df_agg


# 시간 변수 추가
train_agg = add_time_hour(train_agg, train)
test_agg = add_time_hour(test_agg, test)

# 장마 변수 추가
train_agg = add_season(train_agg, train)
test_agg = add_season(test_agg, test)

# 시간 관련 피처 생성
train_agg = add_sin_cos_features(train_agg, 'ef_hour')
test_agg = add_sin_cos_features(test_agg, 'ef_hour')

# 타겟 변수 추가
train_agg = add_target_variable(train_agg, train)


In [20]:
train_agg

,ef_datetime,stn4contest,v01_mean,v01_max,v01_min,v02_mean,v02_max,v02_min,v03_mean,v03_max,...,recent_v06,recent_v07,recent_v08,recent_v09,recent_dh,ef_hour,season,ef_hour_sin,ef_hour_cos,vv
0,2001-05-01 12:00:00,1,0.0,0,0,0.0,0,0,0.0,0,...,0,0,0,0,3,12,0,-4.338837e-01,-0.900969,0.0
1,2001-05-01 12:00:00,2,0.0,0,0,0.0,0,0,0.0,0,...,0,0,0,0,3,12,0,-4.338837e-01,-0.900969,0.0
2,2001-05-01 12:00:00,3,0.0,0,0,0.0,0,0,0.0,0,...,0,0,0,0,3,12,0,-4.338837e-01,-0.900969,0.0
3,2001-05-01 12:00:00,4,0.0,0,0,0.0,0,0,0.0,0,...,0,0,0,0,3,12,0,-4.338837e-01,-0.900969,0.0
4,2001-05-01 12:00:00,5,0.0,0,0,0.0,0,0,0.0,0,...,0,0,0,0,3,12,0,-4.338837e-01,-0.900969,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77518,2003-10-10 21:00:00,16,9.0,9,9,7.0,7,7,7.0,7,...,2,1,0,0,240,21,0,-2.449294e-16,1.000000,0.0
77519,2003-10-10 21:00:00,17,8.0,8,8,6.0,6,6,6.0,6,...,4,0,0,0,240,21,0,-2.449294e-16,1.000000,0.0
77520,2003-10-10 21:00:00,18,9.0,9,9,7.0,7,7,5.0,5,...,0,0,0,0,240,21,0,-2.449294e-16,1.000000,0.8
77521,2003-10-10 21:00:00,19,8.0,8,8,8.0,8,8,6.0,6,...,0,0,0,0,240,21,0,-2.449294e-16,1.000000,0.1


In [21]:
# train/test split

# test 예측
X_train = train_agg.drop(columns=['vv', 'ef_datetime', 'ef_hour'])
y_train = train_agg['vv']
X_test = test_agg.drop(columns=['ef_datetime', 'ef_hour'])


# 데이터 스케일링
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [32]:
# 모델 학습
model = RandomForestRegressor(bootstrap=False, max_depth=8, max_features='log2', n_estimators=200, random_state = 30)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [33]:
pred_df = pd.DataFrame({
    'ef_datetime': test_agg['ef_datetime'],
    'stn4contest': test_agg['stn4contest'],
    'pred': y_pred
})

test_merged = pd.merge(test, pred_df, on=['ef_datetime', 'stn4contest'], how='left')

test_merged = test_merged.dropna(subset=['pred'])

# class_interval 만들기
test_merged['pred_class'] = test_merged['pred'].apply(make_class)

In [26]:
test_merged

,stn4contest,dh,ef_year,ef_month,ef_day,ef_hour,v01,v02,v03,v04,...,v06,v07,v08,v09,class_interval,fc_datetime,ef_datetime,season,pred,pred_class
0,31,3,2004,5,1,12,68,46,13,1,...,0,0,0,0,NaN,2004-05-01 09:00:00,2004-05-01 12:00:00,0,0.130104,1
1,31,6,2004,5,1,15,83,66,26,5,...,0,0,0,0,NaN,2004-05-01 09:00:00,2004-05-01 15:00:00,0,0.243661,2
2,31,9,2004,5,1,18,17,6,0,0,...,0,0,0,0,NaN,2004-05-01 09:00:00,2004-05-01 18:00:00,0,0.057395,0
3,31,12,2004,5,1,21,0,0,0,0,...,0,0,0,0,NaN,2004-05-01 09:00:00,2004-05-01 21:00:00,0,0.004002,0
4,31,15,2004,5,2,0,0,0,0,0,...,0,0,0,0,NaN,2004-05-01 09:00:00,2004-05-02 00:00:00,0,0.003967,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121995,35,228,2004,10,10,9,4,4,4,2,...,0,0,0,0,NaN,2004-09-30 21:00:00,2004-10-10 09:00:00,0,0.279329,2
121996,35,231,2004,10,10,12,8,6,6,5,...,0,0,0,0,NaN,2004-09-30 21:00:00,2004-10-10 12:00:00,0,0.251185,2
121997,35,234,2004,10,10,15,8,6,6,5,...,0,0,0,0,NaN,2004-09-30 21:00:00,2004-10-10 15:00:00,0,0.251185,2
121998,35,237,2004,10,10,18,10,8,8,8,...,2,0,0,0,NaN,2004-09-30 21:00:00,2004-10-10 18:00:00,0,1.055237,4


In [34]:
test2 = pd.read_csv('rainfall_test.csv')

In [35]:
# NaN 처리
test2['rainfall_test.class_interval'].fillna(test_merged['pred_class'], inplace=True)
test2['rainfall_test.class_interval'] = test2['rainfall_test.class_interval'].astype(int)

In [36]:
test2['rainfall_test.class_interval'].unique()

array([   1,    2,    0,    6,    5,    4,    7, -999,    3,    8,    9])

In [37]:
test2.to_csv('240480.csv', index = False)